In [ ]:


##10
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
import random

# Grid environment setup
GRID_SIZE = 6
ACTIONS = ['U', 'D', 'L', 'R']
ACTION_MAP = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}

# Task generator
def create_mail_sorting_task():
    start = (0, 0)
    goal = (np.random.randint(1, GRID_SIZE), np.random.randint(1, GRID_SIZE))
    penalties = set()
    while len(penalties) < 3:
        p = (np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE))
        if p != goal and p != start:
            penalties.add(p)
    obstacles = set()
    while len(obstacles) < 2:
        o = (np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE))
        if o != goal and o != start and o not in penalties:
            obstacles.add(o)
    return {'start': start, 'goal': goal, 'penalties': penalties, 'obstacles': obstacles}

# Environment constraints
def is_valid(state, obstacles):
    x, y = state
    return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and state not in obstacles

# Step transition
def step(state, action, task):
    dx, dy = ACTION_MAP[action]
    next_state = (state[0] + dx, state[1] + dy)
    if not is_valid(next_state, task['obstacles']):
        next_state = state
    reward = -1
    if next_state == task['goal']:
        reward = 10
    elif next_state in task['penalties']:
        reward = -5
    return next_state, reward

# Initialize Q-table
def initialize_Q():
    return {(i, j): {a: 0 for a in ACTIONS} for i in range(GRID_SIZE) for j in range(GRID_SIZE)}

# Train agent with Q-learning
def train_q_learning(task, episodes=200, alpha=0.1, gamma=0.9, epsilon=0.2):
    Q = initialize_Q()
    for _ in range(episodes):
        state = task['start']
        for _ in range(100):
            action = np.random.choice(ACTIONS) if np.random.rand() < epsilon else max(Q[state], key=Q[state].get)
            next_state, reward = step(state, action, task)
            best_next = max(Q[next_state].values())
            Q[state][action] += alpha * (reward + gamma * best_next - Q[state][action])
            if next_state == task['goal']:
                break
            state = next_state
    return Q

# Visualize with heatmap of Q-values
def visualize_policy_heatmap(Q, task, title="Mail Sorting Agent Q-Value Heatmap"):
    fig, ax = plt.subplots(figsize=(6, 6))
    cmap = cm.get_cmap("coolwarm")
    ax.set_xticks(np.arange(GRID_SIZE + 1))
    ax.set_yticks(np.arange(GRID_SIZE + 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            state = (i, j)
            x, y = j, GRID_SIZE - i - 1
            if state == task['goal']:
                color = 'green'
            elif state == task['start']:
                color = 'red'
            elif state in task['penalties']:
                color = 'blue'
            elif state in task['obstacles']:
                color = 'gray'
            else:
                color = 'white'
            rect = patches.Rectangle((x, y), 1, 1, linewidth=1, edgecolor='black', facecolor=color)
            ax.add_patch(rect)
            if state not in task['obstacles']:
                q_values = Q[state]
                # Normalize Q-values for color mapping
                ax.add_patch(patches.Rectangle((x + 0.4, y + 0.7), 0.2, 0.2, color=cmap((q_values['U'] + 10) / 20)))
                ax.add_patch(patches.Rectangle((x + 0.4, y + 0.1), 0.2, 0.2, color=cmap((q_values['D'] + 10) / 20)))
                ax.add_patch(patches.Rectangle((x + 0.1, y + 0.4), 0.2, 0.2, color=cmap((q_values['L'] + 10) / 20)))
                ax.add_patch(patches.Rectangle((x + 0.7, y + 0.4), 0.2, 0.2, color=cmap((q_values['R'] + 10) / 20)))
    plt.title(title)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

# Run the simulation
task = create_mail_sorting_task()
Q = train_q_learning(task)
visualize_policy_heatmap(Q, task)

Lab 10 Task

##10 code

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm

# --- Environment Setup ---
GRID_SIZE = 8  # Increased grid size to fit goals (e.g., (6,7))
ACTIONS = ['U', 'D', 'L', 'R']
ACTION_MAP = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}

# Mail categories and their drop zones (goals)
MAIL_CATEGORIES = {
    'Express': (5, 5),
    'International': (2, 4),
    'Regular': (6, 7)
}

NUM_ROBOTS = 3  # Multiple robots

# --- Task generator ---
def create_mail_sorting_task():
    # Start locations for each robot (could also randomize)
    starts = [(0, 0), (0, GRID_SIZE - 1), (GRID_SIZE - 1, 0)]

    # Random obstacles and shelves
    obstacles = set()
    while len(obstacles) < 5:
        o = (np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE))
        if o not in starts and o not in MAIL_CATEGORIES.values():
            obstacles.add(o)

    penalties = set()
    while len(penalties) < 3:
        p = (np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE))
        if p not in obstacles and p not in starts and p not in MAIL_CATEGORIES.values():
            penalties.add(p)

    return {
        'starts': starts,
        'obstacles': obstacles,
        'penalties': penalties,
        'goals': MAIL_CATEGORIES
    }

# --- Check valid move ---
def is_valid(state, obstacles, other_robots):
    x, y = state
    if not (0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE):
        return False
    if state in obstacles or state in other_robots:
        return False
    return True

# --- Step transition for a robot ---
def step(state, action, task, other_robots):
    dx, dy = ACTION_MAP[action]
    next_state = (state[0] + dx, state[1] + dy)
    if not is_valid(next_state, task['obstacles'], other_robots):
        next_state = state  # invalid move, stay in place

    reward = -1  # step cost

    # No immediate reward for penalties or obstacles since they are blocked
    # Penalties could be thought of as locations to avoid stepping on

    # No penalties if blocked; but if we want penalties inside free cells:
    if next_state in task['penalties']:
        reward = -5

    return next_state, reward

# --- Initialize Q-tables ---
def initialize_Q():
    # For each robot, a Q-table keyed by (x,y) state and action
    Qs = []
    for _ in range(NUM_ROBOTS):
        Q = {(i, j): {a: 0 for a in ACTIONS} for i in range(GRID_SIZE) for j in range(GRID_SIZE)}
        Qs.append(Q)
    return Qs

# --- Training multiple robots ---
def train_multi_robots(task, episodes=300, alpha=0.1, gamma=0.9, epsilon=0.2):
    Qs = initialize_Q()
    mail_categories = list(MAIL_CATEGORIES.keys())

    # Round robin assignment of mail categories to robots
    for ep in range(episodes):
        # Assign each robot a mail category in round-robin fashion
        assigned_mails = [mail_categories[(ep + i) % len(mail_categories)] for i in range(NUM_ROBOTS)]

        states = task['starts'][:]  # Current positions of robots
        done = [False]*NUM_ROBOTS

        for step_idx in range(100):
            # Store next states to avoid collisions
            next_positions = [None]*NUM_ROBOTS

            for r in range(NUM_ROBOTS):
                if done[r]:
                    next_positions[r] = states[r]
                    continue

                current_state = states[r]
                Q = Qs[r]
                mail_cat = assigned_mails[r]
                goal = task['goals'][mail_cat]

                # Epsilon-greedy action selection
                if np.random.rand() < epsilon:
                    action = np.random.choice(ACTIONS)
                else:
                    # Pick max Q for current state
                    action = max(Q[current_state], key=Q[current_state].get)

                # Collect other robots positions excluding current robot
                other_robots = [states[i] for i in range(NUM_ROBOTS) if i != r]

                next_state, reward = step(current_state, action, task, other_robots)

                # If next_state is the goal for that robot's mail category
                if next_state == goal:
                    reward = 20  # bigger reward for delivery
                    done[r] = True

                # Update Q-table
                best_next = max(Q[next_state].values())
                Q[current_state][action] += alpha * (reward + gamma * best_next - Q[current_state][action])

                next_positions[r] = next_state

            # Check collisions - if two robots want to move to same cell, block move
            positions_set = set()
            for idx, pos in enumerate(next_positions):
                if pos in positions_set:
                    # Collision detected - robot stays put
                    next_positions[idx] = states[idx]
                else:
                    positions_set.add(pos)

            states = next_positions

            # If all robots done, end episode
            if all(done):
                break

        # Mid-episode dynamic obstacle rearrangement every 50 episodes
        if ep > 0 and ep % 50 == 0:
            # Change obstacles: randomly add or remove obstacles
            new_obstacles = set()
            while len(new_obstacles) < 5:
                o = (np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE))
                if o not in task['starts'] and o not in task['goals'].values():
                    new_obstacles.add(o)
            task['obstacles'] = new_obstacles

            # Optionally reset some Q-values for new obstacle states to force adaptation
            for r in range(NUM_ROBOTS):
                Q = Qs[r]
                for obs in new_obstacles:
                    # Reset Q-values for obstacle states to zero
                    Q[obs] = {a: 0 for a in ACTIONS}

    return Qs, task

# --- Visualization ---
def visualize_multi_agents(Qs, task, assigned_mails=None, states=None, title="Multi-Robot Mail Sorting"):
    fig, ax = plt.subplots(figsize=(8,8))
    cmap = cm.get_cmap("coolwarm")

    ax.set_xticks(np.arange(GRID_SIZE + 1))
    ax.set_yticks(np.arange(GRID_SIZE + 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True)

    # Draw cells
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            state = (i, j)
            x, y = j, GRID_SIZE - i - 1

            if state in task['goals'].values():
                # Color goals by category
                for cat, pos in task['goals'].items():
                    if pos == state:
                        if cat == 'Express':
                            color = 'green'
                        elif cat == 'International':
                            color = 'orange'
                        else:
                            color = 'purple'
            elif state in task['starts']:
                color = 'red'
            elif state in task['penalties']:
                color = 'blue'
            elif state in task['obstacles']:
                color = 'gray'
            else:
                color = 'white'

            rect = patches.Rectangle((x, y), 1, 1, linewidth=1, edgecolor='black', facecolor=color)
            ax.add_patch(rect)

    # Draw robots
    if states is not None:
        for r in range(NUM_ROBOTS):
            state = states[r]
            x, y = state[1], GRID_SIZE - state[0] -1
            circ = patches.Circle((x+0.5, y+0.5), 0.3, color='cyan', alpha=0.8)
            ax.add_patch(circ)
            ax.text(x+0.5, y+0.5, str(r+1), color='black', weight='bold', ha='center', va='center')

    plt.title(title)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

# --- Run simulation ---
task = create_mail_sorting_task()
Qs, task = train_multi_robots(task)

# Test run: show robot positions after training, assign mails round-robin for visualization
assigned_mails = [list(MAIL_CATEGORIES.keys())[i % len(MAIL_CATEGORIES)] for i in range(NUM_ROBOTS)]
states = task['starts'][:]  # start states

# Visualize final state (no actions here, just initial)
visualize_multi_agents(Qs, task, assigned_mails, states)